# How to Use Warboy From Start To Finish

YOLOv8n 모델을 예시로 Warboy를 사용하는 방법을 보여주는 튜토리얼 주피터 노트북입니다.


## Prerequisites

### Make Python Environment

이 주피터 노트북의 코드들은 Python 3.9 이상의 환경이 필요합니다. 이미 해당하는 Python 환경이 있다면 이 단계를 건너뛸 수 있습니다. 만약 없다면 Conda를 통해 Python 환경을 새롭게 만들 수 있습니다.

Conda가 설치되어 있지 않는 경우, 아래의 명령어를 통해 Miniconda를 설치할 수 있습니다.

```console
$ wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
$ sh ./Miniconda3-latest-Linux-x86_64.sh
$ rm -rf Miniconda3-latest-Linux-x86_64.sh
$ source ~/.bashrc
```


Miniconda를 설치한 후, 다음의 명령어를 사용해 새로운 Python 3.9 환경을 만들 수 있습니다.

```console
$ conda create -n furiosa-3.9 python=3.9
$ conda activate furiosa-3.9
```


### Install Driver, Firmware, and Runtime packages

Warboy를 사용하기 위해서는 가장 먼저, Warboy NPU에 맞는 드라이버, 펌웨어, 런타임 패키지를 설치해야 합니다. 이를 위해서는 우선 APT 서버를 설정해야 하며, 그 방법은 [Korean](https://developer.furiosa.ai/docs/latest/ko/software/installation.html) 또는 [English](https://developer.furiosa.ai/docs/latest/en/software/installation.html)에서 확인할 수 있습니다.

APT 서버 설정을 완료한 후, 아래의 명령어를 통해 드라이버, 펌웨어, 런타임 패키지를 설치할 수 있습니다.

```console
$ sudo apt-get update && sudo apt-get install -y furiosa-driver-warboy furiosa-libnux
```


패키지 설치를 완료한 후, 아래의 명령어를 통해 NPU 장치가 정상적으로 인식되는지 확인할 수 있습니다.

```console
$ sudo apt-get install -y furiosa-toolkit
$ furiosactl info --format full
```


### Install Furiosa Python SDK

Furiosa Python SDK는 pip를 통해 설치할 수 있습니다. SDK에 대한 자세한 내용은 [Korean](https://furiosa-ai.github.io/docs/latest/ko/) 또는 [English](https://furiosa-ai.github.io/docs/latest/en/)에서 확인할 수 있습니다.

```console
$ pip install 'furiosa-sdk[full]'
```

### Install Datasets

이미 dataset을 다운로드 했다면 이 단계를 건너뛸 수 있지만, `CHECK` 표시가 되어있는 부분에서 dataset의 경로가 갖고 있는 dataset의 경로와 일치하는지 확인해주세요.


이 튜토리얼에서는 COCO dataset을 사용할 것이며, 아래의 명령어를 통해 다운로드 할 수 있습니다.

```console
./coco2017.sh
```

이 명령어를 실행하면 COCO dataset이 `datasets/coco` 경로에 저장됩니다.


### Install required packages

아래의 명령어를 통해 필수 패키지를 설치할 수 있습니다.

```console
$ pip install -r requirements.txt
```


또한, 주피터 노트북에서 `asyncio.run()`을 사용하기 위해서는 `nest_asyncio` 패키지를 설치해야 합니다. 아래의 명령어를 통해 설치할 수 있습니다.

```console
$ pip install nest_asyncio
```


### Build Yolo Decoders

이 프로젝트에는, YOLOv8n 모델을 위한 decoder가 일부분 C++로 구현되어 있습니다. 아래의 명령어를 통해 이를 빌드할 수 있습니다.

```console
$ ./build.sh
```


### Install the Project

이 프로젝트를 모듈로 설치하기 위해서는 아래의 명령어를 사용합니다.

```console
$ pip install .
```

이를 통해 이 프로젝트를 설치하게 되면, `warboy-vision` command-line tool을 사용할 수 있게 됩니다. 아래의 명령어를 통해 자세한 사항을 확인할 수 있습니다.

```console
$ warboy-vision --help
```



### Import Modules


In [ ]:
import asyncio
import glob
import imghdr
import os
import random
from pathlib import Path

import cv2
import nest_asyncio
import numpy as np
import onnx
import torch
from tqdm import tqdm
from ultralytics import YOLO


## Prepare Model

Warboy에서 모델을 실행하기 위해서는 quantized ONNX 모델이 필요합니다.


### Download YOLOv8n Weights

가장 먼저, 실행하고자 하는 모델의 weight가 필요합니다. 이 튜토리얼에서는 YOLOv8n 모델을 사용할 것이며, 이미 weight 파일을 다운로드 했다면 이 단계를 건너뛸 수 있습니다.

아래의 명령어를 통해 YOLOv8n 모델의 weight를 다운로드 할 수 있습니다.

```console
wget https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt
```


### Export ONNX

먼저, YOLOv8n 모델을 ONNX 형식으로 export합니다.


In [ ]:
weight_file = "yolov8n.pt"  # CHECK you may change this to your own weight file
onnx_path = "../models/onnx/object_detection/yolov8n.onnx"  # CHECK you may change this to your own path
input_shape = (1, 3, 640, 640)

print(f"Load PyTorch Model from {weight_file}...")
if os.path.dirname(onnx_path) != "" and not os.path.exists(
    os.path.dirname(onnx_path)
):
    os.makedirs(os.path.dirname(onnx_path))

# Load the PyTorch model in inference mode
torch_model = YOLO(weight_file).model.eval()

print(f"Export ONNX {onnx_path}...")
dummy_input = torch.randn(input_shape).to(torch.device("cpu"))

torch.onnx.export(
    torch_model,
    dummy_input,
    onnx_path,
    opset_version=13,
    input_names=["images"],
    output_names=["outputs"],
)


Quantization을 하기에 앞서, YOLO 모델들의 경우, channel 축으로 진행하는 concat 연산자에서 quantization 이후 정확도가 떨어지는 문제가 발생합니다. 이를 해결하기 위해 모델을 수정해 decoding 부분을 제거하고, 이후 연산은 따로 진행해주어야 합니다.


In [ ]:
onnx_path = "../models/onnx/object_detection/yolov8n.onnx"  # CHECK you may change this to your own path

from onnx.utils import Extractor
from src.warboy.tools.utils import get_onnx_graph_info

onnx_graph = onnx.load(onnx_path)
input_to_shape, output_to_shape = get_onnx_graph_info(
    "object_detection", "yolov8n", onnx_path, None
)

edited_graph = Extractor(onnx_graph).extract_model(
    input_names=list(input_to_shape), output_names=list(output_to_shape)
)

for value_info in edited_graph.graph.input:
    del value_info.type.tensor_type.shape.dim[:]
    value_info.type.tensor_type.shape.dim.extend(
        input_to_shape[value_info.name]
    )
for value_info in edited_graph.graph.output:
    del value_info.type.tensor_type.shape.dim[:]
    value_info.type.tensor_type.shape.dim.extend(
        output_to_shape[value_info.name]
    )

print(f"Export edited ONNX >> {onnx_path}")
onnx.save(edited_graph, onnx_path)

### Quantize Model

ONNX 모델이 준비되었다면, 양자화(quantization)를 진행합니다. 양자화는 높은 정밀도(주로 FP32)를 지닌 딥러닝 모델을 낮은 정밀도(Warboy에서는 INT8)로 변환해 모델 사이즈를 축소하여 메모리 비용을 줄이고, 추론 속도를 향상시키는 기술입니다.


양자화 단계에서는 calibration을 위한 dataset이 필요합니다. calibraion dataset은 calibration range를 결정하기 위해 사용되며, 이 튜토리얼에서는 COCO dataset을 사용할 것입니다.


In [ ]:
calibration_data_path = "../../datasets/coco/val2017"   # CEHCK you may change this path to your own path
num_calibration_data = 100

calibration_dataset = []

datas = glob.glob(calibration_data_path + "/**", recursive=True)
datas = random.choices(datas, k=min(num_calibration_data, len(datas)))

for data in datas:
    if os.path.isdir(data) or imghdr.what(data) is None:
        continue
    calibration_dataset.append(data)
print(calibration_dataset)


이제 양자화를 진행해봅시다. calibration 방법과 calibraion data의 개수는 변경할 수 있습니다. 양자화 및 calibraion의 자세한 내용은 [Korean](https://developer.furiosa.ai/docs/latest/ko/software/quantization.html) 또는 [English](https://developer.furiosa.ai/docs/v0.5.0/en/advanced/quantization.html) 문서에서 확인할 수 있습니다.

YOLO 모델의 경우, input data를 640x640으로 resize해서 사용할 수 있습니다. 이를 위해 전처리를 진행하는데, 관련 코드는 `src/warboy/yolo/preprocess.py` 파일의 `YoloPreProcessor`에서 확인할 수 있습니다.


In [ ]:
onnx_path = "../models/onnx/object_detection/yolov8n.onnx"  # CHECK you may change this to your own path
onnx_i8_path = "../models/quantized_onnx/object_detection/yolov8n_i8.onnx"  # CHECK you may change this to your own path
calibration_method = "SQNR_ASYM"
use_model_editor = True

if not os.path.exists(os.path.dirname(onnx_i8_path)):
    os.makedirs(os.path.dirname(onnx_i8_path))

if not os.path.exists(onnx_path):
    raise FileNotFoundError(f"{onnx_path} is not found!")

from furiosa.optimizer import optimize_model
from furiosa.quantizer import (
    CalibrationMethod,
    Calibrator,
    ModelEditor,
    TensorType,
    get_pure_input_names,
    quantize,
)

new_shape = input_shape[2:]
onnx_model = onnx.load(onnx_path)
onnx_model = optimize_model(
    model=onnx_model,
    opset_version=13,
    input_shapes={"images": input_shape},
)

calibrator = Calibrator(
    onnx_model, CalibrationMethod._member_map_[calibration_method]
)

from src.warboy.yolo.preprocess import YoloPreProcessor

preprocessor = YoloPreProcessor(new_shape=new_shape, tensor_type="float32")

for calibration_data in tqdm(
    calibration_dataset, desc="calibration..."
):
    input_img = cv2.imread(calibration_data)
    input_, _ = preprocessor(input_img)
    calibrator.collect_data([[input_]])

if use_model_editor:
    editor = ModelEditor(onnx_model)
    input_names = get_pure_input_names(onnx_model)

    for input_name in input_names:
        editor.convert_input_type(input_name, TensorType.UINT8)

calib_range = calibrator.compute_range()
quantized_model = quantize(onnx_model, calib_range)

with open(onnx_i8_path, "wb") as f:
    f.write(bytes(quantized_model))

print(f"Quantization completed >> {onnx_i8_path}")

## Run Inference

이 튜토리얼에서는 YOLOv8n 모델을 통해 COCO dataset에 대한 inference를 진행한 후, 성능을 평가할 것입니다.

다만, 이 튜토리얼에서는 간단하게 inference를 진행하는 방법을 보여주겠지만, 최적화된 추론을 진행하고 싶다면 `src/warboy/utils/process_pipeline.py`와 `src/warboy/runtime/warboy_runtime.py` 파일을 참고해주세요.


In [ ]:
dataset = "../../datasets/coco/val2017" # CHECK you may change this path to your own path
annotation = "../../datasets/coco/annotations/instances_val2017.json"   # CHECK you may change this path to your own path
onnx_i8_path = "../models/quantized_onnx/object_detection/yolov8n_i8.onnx"  # CHECK you may change this to your own path

앞서 설명한 것처럼, YOLOv8n 모델은 640x640으로 resize된 이미지를 입력으로 받기 때문에, 앞서 사용한 것과 동일한 YoloPreProcessor를 사용해 전처리를 진행합니다.

또한, COCO dataset을 로드하기 위해 `MSCOCODataLoader`를 사용하는데, 관련 코드는 `src/test_scenarios/utils.py`에서 확인할 수 있습니다.


In [ ]:
from src.warboy.yolo.preprocess import YoloPreProcessor
from src.test_scenarios.utils import MSCOCODataLoader

preprocessor = YoloPreProcessor(
    new_shape=input_shape[2:],
    tensor_type="uint8"
)

data_loader = MSCOCODataLoader(
    Path(dataset),
    Path(annotation),
    preprocessor,
    input_shape,
)


마지막으로, inference output을 후처리하기 위한 postprocessor가 필요합니다. Postprocessor는 inference 결과를 decode해 바운딩 박스를 그리거나 계산합니다.

만약 바운딩 박스가 그려진 이미지를 최종적으로 만들고 싶다면, `src/warboy/yolo/postprocess.py` 파일의 `ObjDetPostprocess`를 사용할 수 있습니다.

다만, 이 튜토리얼에서는 mAP를 평가할 것이기 때문에 이미지를 그리는 과정은 생략할 수 있습니다. 이에 따라 postprocessor로 `src/warboy/yolo/decoder.py` 파일의 `object_detection_anchor_decoder`를 사용할 것입니다.


In [ ]:
from src.warboy.yolo.anchor_process import object_detection_anchor_decoder

postprocessor = object_detection_anchor_decoder(
    model_name="yolov8n",
    conf_thres=0.001,   # confidence threshold
    iou_thres=0.7,  # NMS IOU threshold
    anchors=[None],
    use_tracker=False
)


`COCOeval` 함수를 이용해 mAP를 평가할 예정이기 때문에, `xyxy2xywh` 함수를 사용해 바운딩 박스를 xyxy 형식에서 xywh 형식으로 변환하고, `YOLO_CATEGORY_TO_COCO_CATEGORY`를 사용해 카테고리를 YOLO 형식에서 COCO 형식으로 변환해야 합니다. 관련 코드는 `src/test_scenarios/utils.py` 파일에서 확인할 수 있습니다.


In [ ]:
from src.test_scenarios.utils import xyxy2xywh, YOLO_CATEGORY_TO_COCO_CATEGORY

async def warboy_inference(model, data_loader, preprocessor, postprocessor):
    async def task(
        runner, data_loader, preprocessor, postprocessor, worker_id, worker_num
    ):
        results = []
        for idx, (img_path, annotation) in enumerate(data_loader):
            if idx % worker_num != worker_id:
                continue

            img = cv2.imread(str(img_path))
            img0shape = img.shape[:2]
            input_, contexts = preprocessor(img)
            preds = await runner.run([input_])

            outputs = postprocessor(preds, contexts, img0shape)[0]

            bboxes = xyxy2xywh(outputs[:, :4])
            bboxes[:, :2] -= bboxes[:, 2:] / 2

            for output, bbox in zip(outputs, bboxes):
                results.append(
                    {
                        "image_id": annotation["id"],
                        "category_id": YOLO_CATEGORY_TO_COCO_CATEGORY[int(output[5])],
                        "bbox": [round(x, 3) for x in bbox],
                        "score": round(output[4], 5),
                    }
                )
        return results

    from furiosa.runtime import create_runner

    worker_num = 16
    async with create_runner(model, worker_num=32, compiler_config={"use_program_loading": True}) as runner:
        results = await asyncio.gather(
            *(
                task(
                    runner,
                    data_loader,
                    preprocessor,
                    postprocessor,
                    idx,
                    worker_num,
                )
                for idx in range(worker_num)
            )
        )
    return sum(results, [])


이제 모든 준비가 끝났으니 inference를 진행해봅시다.


In [ ]:
nest_asyncio.apply()

results = asyncio.run(
    warboy_inference(onnx_i8_path, data_loader, preprocessor, postprocessor)
)


최종적으로, 모델의 성능을 평가하기 위해 `pycocotools.cocoeval` 모듈의 `COCOeval` 함수를 사용해 mAP를 평가합니다.

In [ ]:
from pycocotools.cocoeval import COCOeval

coco_result = data_loader.coco.loadRes(results)
coco_eval = COCOeval(data_loader.coco, coco_result, "bbox")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

print("mAP: ", coco_eval.stats[0])


## NPU Profiling

Furiosa SDK에서는 모델의 NPU 성능을 분석하기 위한 프로파일링 도구를 제공합니다. 이 도구를 사용하여 모델의 각 연산에 소요되는 시간을 측정하고, bottleneck이 발생하는 지점을 찾아낼 수도 있습니다.

아래의 코드를 실행하면, `tutorials/models/trace` 디렉토리에 trace 파일이 저장됩니다. 이 파일은 Chrome 웹 브라우저의 Trace Event Profiling Tool (chrome://tracing)을 사용하여 시각화할 수 있습니다. 이를 통해 모델의 성능을 이해하고 더 나아가 필요에 따라 최적화까지 진행해볼 수 있습니다.


`OpenTelemetry trace error occurred. cannot send span to the batch span processor because the channel is full` 라는 경고 메시지가 발생할 수 있지만, 무시해도 괜찮은 경고입니다.


In [ ]:
input_shape = (1, 3, 640, 640)
task = "object_detection"
device = "warboy(1)*1"
model = "yolov8n"
onnx_i8_path = "../models/quantized_onnx/object_detection/yolov8n_i8.onnx"  # CHECK you may change this to your own path

from furiosa.runtime.sync import create_runner
from furiosa.runtime.profiler import profile

input_shape = input_shape
trace_dir = os.path.join(
    "../models/trace", task
)  # CHECK you may change this to your own path

if not os.path.exists(trace_dir):
    os.makedirs(trace_dir)

trace_file = os.path.join(trace_dir, model + "_" + device + ".log")
dummy_input = np.uint8(np.random.randn(*input_shape))

with open(trace_file, mode="w") as tracing_file:
    with profile(file=tracing_file) as profiler:
        with create_runner(
            onnx_i8_path, device=device, compiler_config={"use_program_loading": True}
        ) as runner:
            for _ in range(30):
                runner.run([dummy_input])